# Punto 4

In [1]:
import pandas as pd
import matplotlib as mlt
import numpy as np
import matplotlib.pyplot as plt
import cv2
import PyQt5

In [2]:
# Para que las gráficas se impriman no en el notebook, sino en una nueva pestaña
%matplotlib qt

In [3]:
# Función para plotear complejos

def fun_ploteo_complejo(mat, indicador, escala, mapa_color = 'gray'):
    '''
    mat  es una matriz compleja a plotear
    indicador = I para intensidad, A para amplitud, P para fase
    escala = 1 para nada, 0 para logarítmica
    '''
    if(indicador =="I"):
        mat = np.abs(mat)**2
    elif(indicador == "A"):
        mat = np.abs(mat)
    elif(indicador == "P"):
        mat = np.angle(mat)

    if (escala == 1):
        plt.figure()
        plt.imshow(mat, cmap = mapa_color)
        plt.colorbar()
        plt.show()
    else:    
        plt.figure()
        plt.imshow( np.log( mat + 0.001), cmap = mapa_color)
        plt.colorbar()
        plt.show()




## Punto 4 a

In [4]:
# Lectura de los datos

a = pd.read_csv('a.csv', header=None)              # Lecutra del .csv como un objeto np.array

# Dado que la información del csv se carga como texto y además los complejos tienen i en vez de j
# se convertirán esos textos a números complejos
a = a.applymap(lambda s: complex(s.replace('i', 'j'))).values       # Tomado de https://stackoverflow.com/questions/18280489/numpy-read-csv-with-complex-number

np.shape(a)

(720, 720)

In [5]:
# Graficar el campo
fun_ploteo_complejo(a ,"I", 0)

# Graficar las fases del campo donde se encuentra la información de la imagen
fun_ploteo_complejo(a ,"P", 1)

# Graficar la transformada de Fourier shifteada
fun_ploteo_complejo(np.fft.fftshift(np.fft.fftn(a) ),"I", 0)


In [6]:
# Crear la onda monocromática
monocromatica = np.ones(np.shape(a),  dtype=complex)

In [18]:
# hacer interferencia
holografia = a+monocromática


#Aquí obtenemos el gráfico de la intensidad a partir de la interferencia de las ondas (desfasadas y pura)
fun_ploteo_complejo(holografia ,"I", 0)

In [ ]:

ffts_holografia = np.fft.fftshift(np.fft.fftn(holografia) )
fun_ploteo_complejo(ffts_holografia,"I", 0)

# Se puede apreciar un patrón en en el centro como dos semiesferas

ffts_holografia_filtrada = ffts_holografia*mask_4a

# Gráfica de la transformada * máscara
fun_ploteo_complejo(ffts_holografia_filtrada, "I", 0)

# Gráfica de la imagen real despues de filtrar la transformada con la máscara
fun_ploteo_complejo(np.fft.fftn(ffts_holografia_filtrada), "I", 0)

In [33]:
#Creación de la máscara

mask_4a = np.ones(np.shape(a), dtype = "uint8") # uint8 tipo de la variable de la matriz (enteros de 8 bit)

# Como mi imagen no es cuadrada, entonces necesito dos coordenadas
# para el centro
coord0 = int(np.shape(a)[0]/2)
coord1 = int(np.shape(a)[1]/2)

print(coord0, coord1)

#Graficar la máscara circular 
cv2.circle(mask_4a, (coord1, coord0), 25, 0, -1)
#arg1  #arg2 = posiciones de columnas #arg 3=pixeles dentro de la máscara # arg4 = que me rellene todo
plt.figure()
plt.imshow(mask_4a)
plt.colorbar() # Para mostrar barra de colores
plt.show()

360 360


In [9]:
#Transformada de la matriz a
fft_a = np.fft.fftn(a)

# Shifteada
fft_a_shifted = np.fft.fftshift(fft_a)

# Aplicando la máscara (en el espacio de frecuencias)
fft_a_filtrada = fft_a_shifted*mask_4a

# Regresando al espacio real aplicando la máscara
a_filtrada = np.fft.ifftn(fft_a_filtrada)

# Gráfica de la máscara
fun_ploteo_complejo(mask_4a, "A", 0)

# Gráfica de la transformada shifteada
fun_ploteo_complejo(fft_a_shifted, "A", 0)
#Grágica de la fase de la transformada
fun_ploteo_complejo(fft_a_shifted, "P", 1, 'viridis')


# Gráfica de la transformada * máscara
fun_ploteo_complejo(fft_a_filtrada, "A", 0)

# Gráfica de la imagen real despues de filtrar la transformada con la máscara
fun_ploteo_complejo(a_filtrada, "A", 1)


In [10]:


plt.figure()
plt.hist(np.real(a))
plt.show()


plt.figure()
plt.hist(np.imag(a))
plt.show()


plt.figure()
plt.hist(np.abs(a))
plt.show()

plt.figure()
plt.hist(np.angle(a))
plt.show()

# Punto 4 c

In [12]:
# Lectura de los datos

war_img = cv2.imread('c.jpeg')              # Lecutra del .csv como un objeto np.array
waly_img = cv2.imread('c_clue.jpeg')
# Dado que la información del csv se carga como texto y además los complejos tienen i en vez de j
# se convertirán esos textos a números complejos


In [13]:
# Veamos las dimensiones de las imágenes
print("dimensiones de la imagen de guerra: " ,np.shape(war_img), "\ndimensiones de waly: ", np.shape(waly_img) )

# Cambiar a RGB
war_img = cv2.cvtColor(war_img, cv2.COLOR_BGR2RGB)
waly_img = cv2.cvtColor(waly_img, cv2.COLOR_BGR2RGB)

# Mostrar imágenes
plt.figure()
plt.imshow(war_img)
plt.show()

plt.figure()
plt.imshow(waly_img)
plt.show()

dimensiones de la imagen de guerra:  (1680, 2600, 3) 
dimensiones de waly:  (64, 64, 3)


Como tenemos imágnes RGB, entonces por facilidad las pasaremos a escala de grices, después si se desea, 
generalizamos para imágenes RGB

In [14]:
# Convertir imágenes RGB a escala de grises
war_gray = cv2.cvtColor(war_img, cv2.COLOR_RGB2GRAY)
waly_gray = cv2.cvtColor(waly_img, cv2.COLOR_RGB2GRAY)

# Mostrar imágenes
plt.figure()
plt.imshow(war_gray, cmap = 'gray')
plt.show()

plt.figure()
plt.imshow(waly_gray, cmap = 'gray')
plt.show()

In [15]:
# Transformadas de Fourier a las imágenes
fft_war = np.fft.fftn(war_gray)
fft_waly = np.fft.fftn(waly_gray)

# Shifteando
ffts_war = np.fft.fftshift(fft_war)
ffts_waly = np.fft.fftshift(fft_waly)

# Graficando las transofrmadas
fun_ploteo_complejo(ffts_war, "I", 0)
fun_ploteo_complejo(ffts_waly, "I", 0)

# Pad

In [26]:
dimension_pad = (np.array(np.shape(war_gray) ) - np.array(np.shape(waly_gray) ) )/2
waly_paded = np.pad(waly_gray, ( (int(dimension_pad[0]), int(dimension_pad[0]) ), (int(dimension_pad[1]), int(dimension_pad[1])) ), mode = 'constant', constant_values = (0,0) )

print(np.shape(waly_paded) )
#plt.imshow(waly_paded)
# Fourier
fft_p_waly = np.fft.fftn(waly_paded)
ffts_p_waly = np.fft.fftshift(fft_p_waly)       # Shift

filtrada = ffts_war * np.conj(ffts_p_waly)      # Sacando complejo conjugado a fft pad de waly y multiplicando  las transformadas


#Graficando
# fft padeado de waly
fun_ploteo_complejo(ffts_p_waly, "I", 0)
# ffts de la imagen original
fun_ploteo_complejo(ffts_war, "I" , 0)
# la multiplicación de las transformadas
fun_ploteo_complejo(filtrada, "I", 0)

# Regresando a al dominio espacial y graficando
fun_ploteo_complejo(np.fft.fftn(filtrada), "I", 1)

(1680, 2600)
